In [60]:
import numpy as np 
import pandas as pd

real_data = pd.read_csv('True.csv')
fake_data = pd.read_csv('Fake.csv') 

In [61]:
fake_data.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [62]:
print(len(real_data))
print(len(fake_data))

# fake데이터와 true데이터의 개수 동일하게 설정
nb_articles = min(len(real_data), len(fake_data))
real_data = real_data[:nb_articles]
fake_data = fake_data[:nb_articles]

print(len(real_data))
print(len(fake_data))

21417
23481
21417
21417


In [63]:
# row에 'is_fake' row 추가하여 값 초기화
real_data['is_fake'] = False
fake_data['is_fake'] = True

In [64]:
real_data.head()

,title,text,subject,date,is_fake
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",False
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",False
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",False
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",False
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",False


In [65]:
fake_data.head()

,title,text,subject,date,is_fake
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",True
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",True
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",True
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",True
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",True


In [66]:
from sklearn.utils import shuffle

data = pd.concat([real_data, fake_data])

# fake, true 데이터를 합치고, 무작위 셔플
data = shuffle(data).reset_index(drop=True)
data.head()

,title,text,subject,date,is_fake
0,Death of a blogger casts shadow over murky Malta,VALLETTA (Reuters) - Daphne Caruana Galizia po...,worldnews,"October 20, 2017",False
1,(Video) Trump Gives BEST ASSESSMENT EVER Of Ob...,Presidential Candidate Donald Trump gets to th...,politics,"Oct 3, 2015",True
2,WOW! PRO-CASTRO LEFTIST: Cubans Fled Castro To...,,politics,"Nov 29, 2016",True
3,FORMER CIA DIRECTOR Admits Guilt of Intel Agen...,Remember former CIA Director Mike Morell s lie...,politics,"Dec 11, 2017",True
4,WATCH: Samantha Bee Shows Just How CRAZY Ted ...,One has to wonder why in the world people woul...,News,"April 5, 2016",True


In [67]:
# 학습데이터 60%, 검증데이터 20%, 테스트데이터 20% 로 분리
train_data, validate_data, test_data = np.split(data.sample(frac=1), [int(.6*len(data)), int(.8*len(data))])

# reset_index하여 각 데이터별 인덱스 정렬
train_data = train_data.reset_index(drop=True)
validate_data = validate_data.reset_index(drop=True)
test_data = test_data.reset_index(drop=True)

del real_data
del fake_data

print("Size of training set : {}".format(len(train_data)))
print("Size of validation set: {}".format(len(validate_data)))
print("Size of testing set: {}".format(len(test_data)))

Size of training set : 25700
Size of validation set: 8567
Size of testing set: 8567


In [9]:
!conda install -y pytorch torchvision cudatoolkit=10.1 -c pytorch
!pip install transformers

Solving environment: ...working... done

# All requested packages already installed.

  Using cached transformers-4.4.2-py3-none-any.whl (2.0 MB)
  Created wheel for sacremoses: filename=sacremoses-0.0.44-py3-none-any.whl size=886084 sha256=4d9b87e869e5d497f3d8e9323b7e7531585e23bf508c7abaa363251b8db1b9f7
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\d6\17\75\f2ed13c472c4cecc14f003401bb45efadca64cc589d4bf3103
Successfully built sacremoses


In [68]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# 쿠다없으면 cpu 사용
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased")
model.config.num_labels = 1

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [69]:
# Freeze the pre trained parameters
for param in model.parameters():
    param.requires_grad = False

# Add three new layers at the end of the network
model.classifier = nn.Sequential(
    nn.Linear(768, 256),
    nn.ReLU(),
    nn.Linear(256, 64),
    nn.ReLU(),
    nn.Linear(64, 2),
    nn.Softmax(dim=1)
)

model = model.to(device)

In [70]:
criterion = nn.MSELoss().to(device)
optimizer = optim.SGD(model.classifier.parameters(), lr=0.01)

In [71]:
def preprocess_text(text):
    parts = []

    text_len = len(text.split(' '))
    delta = 300
    max_parts = 5
    nb_cuts = int(text_len / delta)
    nb_cuts = min(nb_cuts, max_parts)
    
    
    for i in range(nb_cuts + 1):
        text_part = ' '.join(text.split(' ')[i * delta: (i + 1) * delta])
        parts.append(tokenizer.encode(text_part, return_tensors="pt", max_length=500).to(device))

    return parts

In [81]:
# 모델 로드
model.load_state_dict(torch.load("Model_BertTokenizer_and_BertForSequenceClassification.pt")) 
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [82]:
total = len(test_data)
number_right = 0
model.eval()
# 서버의 메모리를 아끼기 위해
# 예측을 실행하기 전에 그라디언트를 정보를 저장하지 않는다.
# torch.no_grad()
with torch.no_grad():
    for idx, row in test_data.iterrows():
        text_parts = preprocess_text(str(row['text']))
        label = torch.tensor([row['is_fake']]).float().to(device)
        
        overall_output = torch.zeros((1,2)).to(device)
        try:
            for part in text_parts:
                if len(part) > 0:
                    overall_output += model(part.reshape(1, -1))[0]
        except RuntimeError:
            print("GPU out of memory, skipping this entry.")
            continue
            
        overall_output = F.softmax(overall_output[0], dim=-1)
            
        result = overall_output.max(0)[1].float().item()
 
        if result == label.item():
            number_right += 1
            
        if idx % print_every == 0 and idx > 0:
            print("{}/{}. Current accuracy: {}".format(idx, total, number_right / idx))
            
print("Accuracy on test data: {}".format(number_right / total))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


300/8567. Current accuracy: 0.9866666666666667
600/8567. Current accuracy: 0.9716666666666667
900/8567. Current accuracy: 0.97
1200/8567. Current accuracy: 0.9691666666666666
1500/8567. Current accuracy: 0.9653333333333334
1800/8567. Current accuracy: 0.9677777777777777
2100/8567. Current accuracy: 0.9666666666666667
2400/8567. Current accuracy: 0.9658333333333333
2700/8567. Current accuracy: 0.9655555555555555
3000/8567. Current accuracy: 0.964
3300/8567. Current accuracy: 0.9645454545454546
3600/8567. Current accuracy: 0.965
3900/8567. Current accuracy: 0.9648717948717949
4200/8567. Current accuracy: 0.9654761904761905
4500/8567. Current accuracy: 0.9657777777777777
4800/8567. Current accuracy: 0.9664583333333333
5100/8567. Current accuracy: 0.9668627450980393
5400/8567. Current accuracy: 0.9668518518518519
5700/8567. Current accuracy: 0.966842105263158
6000/8567. Current accuracy: 0.9656666666666667
6300/8567. Current accuracy: 0.966031746031746
6600/8567. Current accuracy: 0.966060

In [110]:
def test(text):
    text_parts = preprocess_text(text)
    overall_output = torch.zeros((1,2)).to(device)
    
    for part in text_parts:
            if len(part) > 0:
                overall_output += model(part.reshape(1, -1))[0]

    overall_output = F.softmax(overall_output[0], dim=-1)
    
    # result에 판별 결과 저장, value에 예측도 저장
    value, result = overall_output.max(0)

    return result

In [111]:
v_right = 0
for idx,row in validate_data.iterrows():
    result = test(validate_data['text'][idx])
    label = torch.tensor([row['is_fake']]).float().to(device)
    
    if result == label.item():
        v_right += 1
    if idx % 1000 == 0 and idx > 0:
        print("{}/{} complete".format(idx,len(validate_data) ) )

print(v_right)

1000/8567 complete
2000/8567 complete
3000/8567 complete
4000/8567 complete
5000/8567 complete
6000/8567 complete
7000/8567 complete
8000/8567 complete
8292


In [113]:
print("검증 데이터에 대한 정확도 : {}%" .format(v_right/len(validate_data)*100) )

검증 데이터에 대한 정확도 : 96.7900081708883%
